In [ ]:
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import tensorflow as tf
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from warnings import filterwarnings
filterwarnings('ignore')
from kerastuner.tuners import RandomSearch

In [ ]:
df=pd.read_csv('../input/rideshare_kaggle.csv') 

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df=df.drop(['id'],axis=1)

In [ ]:
df.head()

In [ ]:
df=df.drop(['timestamp','datetime','windGustTime','temperatureHigh','temperatureHighTime','temperatureLow','temperatureLowTime',
           'apparentTemperatureHigh','apparentTemperatureHighTime','apparentTemperatureLow','apparentTemperatureLowTime',
            'sunriseTime','sunsetTime','precipIntensityMax','uvIndexTime','temperatureMin','temperatureMinTime','temperatureMax',
            'temperatureMaxTime','apparentTemperature','apparentTemperatureMin','apparentTemperatureMinTime','apparentTemperatureMax'
           ,'apparentTemperatureMaxTime','long_summary','icon','product_id','timezone','visibility.1','destination'],axis=1)

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.describe(include='object')

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:

plt.figure(figsize=(25, 10))

sns.heatmap(df.isnull(), cbar=False)

plt.show()

In [ ]:
df=df[df['price'].isnull()==False]

In [ ]:
df.isnull().sum()

In [ ]:

plt.figure(figsize=(25, 10))

sns.heatmap(df.isnull(), cbar=False)

plt.show()

In [ ]:
df.shape

In [ ]:
num_col=df.select_dtypes(include='number')

In [ ]:
num_col.shape

In [ ]:
cat_col=df.select_dtypes(include='object')

In [ ]:
cat_col.shape

In [ ]:
cat_col.head()

In [ ]:
plt.figure(figsize=(60,55))
plt.rcParams.update({'font.size': 40})
j=1
for column in num_col:
    plt.subplot(8,3,j)
    sns.distplot(num_col[column])
    plt.tight_layout()
    j=j+1

In [ ]:
plt.figure(figsize=(60,55))
plt.rcParams.update({'font.size': 40})
j=1
for column in num_col:
    if column=='price':
        continue
    else:
        plt.subplot(8,3,j)
        sns.scatterplot(x=num_col[column],y=num_col['price'])
        plt.tight_layout()
        j=j+1

In [ ]:
cat_col1=cat_col.copy()

In [ ]:
cat_col['price']=df['price']

In [ ]:
plt.figure(figsize=(70,69))
plt.rcParams.update({'font.size': 40})
j=1
for column in cat_col:
    if column=='price':
        continue
    else:
        plt.subplot(8,1,j)
        sns.boxplot(x=cat_col[column],y=cat_col['price'])
        plt.tight_layout()
        j=j+1

In [ ]:
corr=num_col.corr()

In [ ]:
# set figure size
plt.figure(figsize=(30, 30))

# set condition to get a strong correlation between the variables
sns.set(font_scale=2)
sns.heatmap(corr[(corr >= 0.7) | (corr <= -0.7)],
            vmax=1.0, vmin=-1.0, linewidths=0.1,
           annot=True, annot_kws={"size": 18}, square=True);

# specify name of the plot
plt.title('Correlation between numeric features')
plt.show()

In [ ]:
plt.figure(figsize=(60,55))
plt.rcParams.update({'font.size': 40})
j=1
for column in num_col:
    plt.subplot(8,3,j)
    sns.boxplot(x=num_col[column])
    plt.tight_layout()
    j=j+1

In [ ]:
data2=df[['price','distance','latitude','temperature','windGust','visibility','dewPoint']]

In [ ]:
#Outlier Treatment
#Outlier Treatment
#Capping the outliers #Winsorization
for i in data2.columns:
    q1=data2[i].quantile(0.25)
    q3=data2[i].quantile(0.75)
    iqr=q3-q1
    ub=q3 + 1.5*iqr
    lb=q1 - 1.5*iqr
    uc=data2[i].quantile(0.99)
    lc=data2[i].quantile(0.01)
    for ind1 in data2[i].index:
        if data2.loc[ind1, i] >ub:            
            data2.loc[ind1, i] =uc
        if data2.loc[ind1, i] < lb:
            data2.loc[ind1, i] =lc

In [ ]:
plt.figure(figsize=(60,55))
plt.rcParams.update({'font.size': 40})
j=1
for column in data2:
    plt.subplot(8,3,j)
    sns.boxplot(x=data2[column])
    plt.tight_layout()
    j=j+1

In [ ]:
for column in data2:
    df[column]=data2[column]

In [ ]:
num_col=df.select_dtypes(include='number')

In [ ]:
plt.figure(figsize=(60,55))
plt.rcParams.update({'font.size': 40})
j=1
for column in num_col:
    plt.subplot(8,3,j)
    sns.boxplot(x=num_col[column])
    plt.tight_layout()
    j=j+1

In [ ]:
cat_col

In [ ]:
df.source.value_counts()

In [ ]:
df.cab_type.value_counts()

In [ ]:
df.name.value_counts()

In [ ]:
df.short_summary.value_counts()

In [ ]:
df=pd.get_dummies(df,columns=['cab_type'],drop_first=True)

In [ ]:
df=pd.get_dummies(df,columns=['short_summary'],drop_first=True)

In [ ]:
df=pd.get_dummies(df,columns=['name'],drop_first=True)

In [ ]:
df=pd.get_dummies(df,columns=['source'],drop_first=True)

In [ ]:
df.head()

# **Base Model**

In [ ]:
data=df.copy()

In [ ]:
X=data.drop(['price'],axis=1)
Y=data['price']
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.3,random_state=48)

In [ ]:
lm=LinearRegression()

In [ ]:
lm.fit(x_train,y_train)

In [ ]:
pred=lm.predict(x_test)

In [ ]:
r2_score(y_test,pred)

In [ ]:
num_col=num_col-num_col.mean()

In [ ]:
 vif=pd.DataFrame()

In [ ]:
vif['VIF']=[variance_inflation_factor(num_col.values,i) for i in range(num_col.shape[1])] 
vif['feature']=num_col.columns 
vif.sort_values('VIF',ascending=False)

In [ ]:
num_col=num_col.drop(['dewPoint'],axis=1)

In [ ]:
vif1=pd.DataFrame()
vif1['VIF']=[variance_inflation_factor(num_col.values,i) for i in range(num_col.shape[1])] 
vif1['feature']=num_col.columns 
vif1.sort_values('VIF',ascending=False)

In [ ]:
num_col=num_col.drop(['month'],axis=1)

In [ ]:
vif1=pd.DataFrame()
vif1['VIF']=[variance_inflation_factor(num_col.values,i) for i in range(num_col.shape[1])] 
vif1['feature']=num_col.columns 
vif1.sort_values('VIF',ascending=False)

In [ ]:
num_col=num_col.drop(['windSpeed'],axis=1)

In [ ]:
vif1=pd.DataFrame()
vif1['VIF']=[variance_inflation_factor(num_col.values,i) for i in range(num_col.shape[1])] 
vif1['feature']=num_col.columns 
vif1.sort_values('VIF',ascending=False)

In [ ]:
num_col=num_col.drop(['pressure'],axis=1)

In [ ]:
vif1=pd.DataFrame()
vif1['VIF']=[variance_inflation_factor(num_col.values,i) for i in range(num_col.shape[1])] 
vif1['feature']=num_col.columns 
vif1.sort_values('VIF',ascending=False)

In [ ]:
num_col=num_col.drop(['precipProbability'],axis=1)

In [ ]:
vif1=pd.DataFrame()
vif1['VIF']=[variance_inflation_factor(num_col.values,i) for i in range(num_col.shape[1])] 
vif1['feature']=num_col.columns 
vif1.sort_values('VIF',ascending=False)

In [ ]:
data=data.drop(['dewPoint','month','windSpeed','pressure','precipProbability'],axis=1)

# **Model-2**

In [ ]:
X=data.drop(['price'],axis=1)
Y=data['price']
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.3,random_state=48)

In [ ]:
lm1=LinearRegression()

In [ ]:
lm1.fit(x_train,y_train)

In [ ]:
pred=lm1.predict(x_test)

In [ ]:
r2_score(y_test,pred)